# 文本分析

自然语言处理 (NLP) 是人工智能 (AI) 的一个分支，用于处理手写以及口头语言。可以使用 NLP 构建从文本或语音中提取语义含义或者使用自然语言发出合理回应的解决方案。

Microsoft Azure *认知服务*包括*文本分析*服务，该服务提供了一些现成的 NLP 功能，包括识别文本中的关键短语和基于情绪对文本分类。

![正在阅读笔记的机器人](./images/NLP.jpg)

例如，假设虚构的 *Margie's Travel* 组织鼓励客户提交关于酒店住宿的评价。使用文本分析服务，可以通过提取关键短语来对评价进行总结，确定哪些评价是正面的，哪些是负面的，或者分析评价文本中提到的已知实体（如地点或人员）。

## 查看评价文档

让我们先来看看顾客留下的一些酒店评价。

这些评价都包含在文本文件中。要查看它们，只需单击位于单元格左侧的“**运行单元格**”(&#9655;) 按钮以运行下方的代码即可。

In [ ]:
import os

# Read the reviews in the /data/reviews folder
reviews_folder = os.path.join('data', 'text', 'reviews')

# Create a collection of reviews with id (file name) and text (contents) properties
reviews = []
for file_name in os.listdir(reviews_folder):
    review_text = open(os.path.join(reviews_folder, file_name)).read()
    review = {"id": file_name, "text": review_text}
    reviews.append(review)

for review_num in range(len(reviews)):
    # print the review text
    print('{}\n{}\n'.format(reviews[review_num]['id'], reviews[review_num]['text']))

## 创建认知服务资源

要分析这些评价中的文本，可以使用**文本分析**认知服务。要使用该服务，你需要在 Azure 订阅中预配**文本分析**或**认知服务**资源（如果仅计划使用这一项服务或者想单独跟踪它的使用情况，则使用文本分析资源；否则，可以使用认知服务资源将文本分析服务与其他认知服务结合使用，使开发人员能够使用单个终结点和密钥来访问它们。）

请按照以下步骤在 Azure 订阅中创建**认知服务**资源（如果还没有该资源）：

> **备注**：如果已有认知服务资源，则只需在 Azure 门户中打开其“**快速入门**”页面，然后将其密钥和终结点复制到下面的单元格中即可。否则，请按照以下步骤创建认知服务资源。

1. 在另一个浏览器标签页中，打开 Azure 门户 (https://portal.azure.com) 并使用 Microsoft 帐户登录。
2. 单击“**&#65291;创建资源**”按钮，搜索“*认知服务*”并以如下设置创建**认知服务**资源：
    - **订阅**： *你的 Azure 订阅*。
    - **资源组**： *选择或创建具有唯一名称的资源组*。
    - **区域**： *选择任何可用区域*：
    - **名称**： *输入一个唯一名称*。
    - **定价层**：中的机器人 S0
    - **我确认我已阅读并理解上述通知**：已选中。
3. 等待部署完成。然后转到认知服务资源，并单击“**概述**”页面上的链接以管理该服务的密钥。你将需要使用终结点和密钥从客户端应用程序连接到认知服务资源。

### 获取认知服务资源的密钥和终结点

要使用认知服务资源，客户端应用程序需要其终结点和身份验证密钥：

1. 进入 Azure 门户，在认知服务资源的“**密钥和终结点**”页面上复制资源的“**Key1**”，并将其粘贴到以下代码中，替换“**YOUR_COG_KEY**”。
2. 复制资源的**终结点**，并将其粘贴到以下代码中，替换“**YOUR_COG_ENDPOINT**”。
3. 通过单击绿色的 <span style="color:green">&#9655;</span> 按钮运行下方单元格中的代码。

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

print('Ready to use cognitive services at {} using key {}'.format(cog_endpoint, cog_key))

## 检测语言
让我们先确定这些评价所采用的语言。

In [ ]:
import os
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials

# Get a client for your text analytics cognitive service resource
text_analytics_client = TextAnalyticsClient(endpoint=cog_endpoint,
                                            credentials=CognitiveServicesCredentials(cog_key))

# Analyze the reviews you read from the /data/reviews folder earlier
language_analysis = text_analytics_client.detect_language(documents=reviews)

# print detected language details for each review
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the language details for this review
    lang = language_analysis.documents[review_num].detected_languages[0]
    print(' - Language: {}\n - Code: {}\n - Score: {}\n'.format(lang.name, lang.iso6391_name, lang.score))

    # Add the detected language code to the collection of reviews (so we can do further analysis)
    reviews[review_num]["language"] = lang.iso6391_name

## 提取关键短语

现在可以分析客户评价中的文本，识别一些能体现客户主要观点的关键短语。

In [ ]:
# # Use the client and reviews you created in the previous code cell to get key phrases
key_phrase_analysis = text_analytics_client.key_phrases(documents=reviews)

# print key phrases for each review
for review_num in range(len(reviews)):
    # print the review id
    print(reviews[review_num]['id'])

    # Get the key phrases in this review
    print('\nKey Phrases:')
    key_phrases = key_phrase_analysis.documents[review_num].key_phrases
    # Print each key phrase
    for key_phrase in key_phrases:
        print('\t', key_phrase)
    print('\n')

关键短语可以帮助你了解每条评价中最重要的观点。例如，如果评价中包含“工作人员乐于助人”或“服务差劲”这样的短语，你就可了解到一些评价者最在意的问题。

## 判断情绪

此功能可用于根据*情绪评分*将评价分为*正面*评价或*负面*评价。同样，你可以使用文本分析服务来做到这一点。

In [ ]:
# Use the client and reviews you created previously to get sentiment scores
sentiment_analysis = text_analytics_client.sentiment(documents=reviews)

# Print the results for each review
for review_num in range(len(reviews)):

    # Get the sentiment score for this review
    sentiment_score = sentiment_analysis.documents[review_num].score

    # classifiy 'positive' if more than 0.5,
    if sentiment_score < 0.5:
        sentiment = 'negative'
    else:
        sentiment = 'positive'

    # print file name and sentiment
    print('{} : {} ({})'.format(reviews[review_num]['id'], sentiment, sentiment_score))

## 提取已知实体

*实体*是指可能在文本中提到的内容，涉及一些人们所熟知的项目类型。例如地点、人员或日期。假设你想了解评价中提到的日期和地点，则可使用下面的代码来查找这些信息。

In [ ]:
# Use the client and reviews you created previously to get named entities
entity_analysis = text_analytics_client.entities(documents=reviews)

# Print the results for each review
for review_num in range(len(reviews)):
    print(reviews[review_num]['id'])
    # Get the named entitites in this review
    entities = entity_analysis.documents[review_num].entities
    for entity in entities:
        # Only print datetime or location entitites
        if entity.type in ['DateTime','Location']:
            link = '(' + entity.wikipedia_url + ')' if entity.wikipedia_id is not None else ''
            print(' - {}: {} {}'.format(entity.type, entity.name, link))

请注意，有些知名实体会具有与之关联的 Wikipedia 页面，在这种情况下，文本分析服务会返回该页面的 URL。

## 了解详细信息

有关文本分析服务的详细信息，请参阅[文本分析服务文档](https://docs.microsoft.com/azure/cognitive-services/text-analytics/)